In [3]:
import requests
import time
import os

In [2]:
def transcribe_yandex(filelink):
    print(filelink)
    # https://cloud.yandex.ru/services/speechkit

    # Укажите ваш API-ключ и ссылку на аудиофайл в Object Storage.
    with open('yandex_api.key', 'r') as file:
        key = file.read().replace('\n', '')
        file.close()

    POST = "https://transcribe.api.cloud.yandex.net/speech/stt/v2/longRunningRecognize"

    body ={
        "config": {
            "specification": {
                "languageCode": "ru-RU",
                "audio_encoding":'LINEAR16_PCM',
                "sample_rate_hertz":8000
            }
        },
        "audio": {
            "uri": filelink
        }
    }

    # Если вы хотите использовать IAM-токен для аутентификации, замените Api-Key на Bearer.
    header = {'Authorization': 'Api-Key {}'.format(key)}

    # Отправить запрос на распознавание.
    req = requests.post(POST, headers=header, json=body)
    data = req.json()
    print(data)
    id = data['id']

    # Запрашивать на сервере статус операции, пока распознавание не будет завершено.
    while True:

        time.sleep(1)

        GET = "https://operation.api.cloud.yandex.net/operations/{id}"
        req = requests.get(GET.format(id=id), headers=header)
        req = req.json()

        if req['done']: break
        #print("Not ready")

    # Показать полный ответ сервера в формате JSON.
    #print("Response:")
    #print(json.dumps(req, ensure_ascii=False, indent=2))

    # Показать только текст из результатов распознавания.
    #print("Text chunks:")
    result = ''
    for chunk in req['response']['chunks']:
        result += chunk['alternatives'][0]['text'] + ' '

    return result

In [4]:
# conver to ogg
file = 'ru.wav'
os_cmd 	= 'ffmpeg -i '
os_cmd += file
os_cmd += ' -f f32le -ar 8000 -acodec pcm_s16le'
os_cmd += ' ' + file[:-4] + '.ogg'
try:
    os.system(os_cmd)
except Exception as e:
    print('ffmpeg converting error:',str(e))
print("now drag'n'drop files to bucket")

now drag'n'drop files to bucket


ffmpeg version 4.4.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 9.4.0 (GCC)
  configuration: --prefix=/home/alex/anaconda3/envs/pycuda --cc=/home/conda/feedstock_root/build_artifacts/ffmpeg_1636205340875/_build_env/bin/x86_64-conda-linux-gnu-cc --disable-doc --disable-openssl --enable-avresample --enable-demuxer=dash --enable-gnutls --enable-gpl --enable-hardcoded-tables --enable-libfreetype --enable-libopenh264 --enable-vaapi --enable-libx264 --enable-libx265 --enable-libaom --enable-libsvtav1 --enable-libxml2 --enable-libvpx --enable-pic --enable-pthreads --enable-shared --disable-static --enable-version3 --enable-zlib --enable-libmp3lame --pkg-config=/home/conda/feedstock_root/build_artifacts/ffmpeg_1636205340875/_build_env/bin/pkg-config
  libavutil      56. 70.100 / 56. 70.100
  libavcodec     58.134.100 / 58.134.100
  libavformat    58. 76.100 / 58. 76.100
  libavdevice    58. 13.100 / 58. 13.100
  libavfilter     7.110.100 /  7.110.100
  libavresample   4.  0

In [5]:
file = file[:-4] + '.ogg'
r = transcribe_yandex('https://storage.yandexcloud.net/voskaudio20211123/' + file)
r

https://storage.yandexcloud.net/voskaudio20211123/ru.ogg
{'done': False, 'id': 'e033ent39tn758a291ni', 'createdAt': '2022-07-01T05:36:51Z', 'createdBy': 'ajeel38rchgt6n77eu0q', 'modifiedAt': '2022-07-01T05:36:51Z'}


'Сегодня ожидается хорошая погода без осадков температура воздуха 9° '